In [6]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder


In [7]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML/Lab4/spam.csv') 


Analysis
To analyze the text data, we have to turn the words into numerical numbers. We have multiple choices to accomplish this step:

1) Binary Term Frequency : count presence(1) or absence(0) for term in document

2) Bag of Words Frequency: captures the frequency of term in document

3) Term Frequency:

4) TFIDF :

in this way, if a term appears frequently in a document, it’s important; if a term appears in many documents, it’s not a unique identifier.

useful links :

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
https://machinelearningmastery.com/gentle-introduction-bag-words-model/
https://towardsdatascience.com/hacking-scikit-learns-vectorizers-9ef26a7170af
https://towardsdatascience.com/top-5-word-tokenizers-that-every-nlp-data-scientist-should-know-45cc31f8e8b9

In [8]:
print("\nData :\n",dataset)
print("\nData statistics\n",dataset.info())


Data :
         v1                                                 v2
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will �_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2 

In [9]:
dataset.dropna(how='any',inplace=True)
print(dataset.isnull().sum())
dataset.drop_duplicates(inplace=True)
print(dataset.duplicated().any())

v1    0
v2    0
dtype: int64
False


In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('all')
import string
import re
import random

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

re.findall(r"[a-z]+", "123ram=*kisanw217-+/")

In [19]:
def stemWords(list1):
    #     stemmer=nltk.stem.PorterStemmer()
    stemmer=nltk.stem.SnowballStemmer('english')
    #     leminizer.lemmatize(word)
    #     stemmer = nltk.stem.ISRIStemmer()
    #     leminizer = nltk.stem.WordNetLemmatizer()
    #     stemmer = nltk.stem.LancasterStemmer()
    
    words=[]
    for text in list1:
        for word in re.findall(r"[a-z]+",text):
            if (len(word)>2 and word not in string.punctuation):
                words.append(stemmer.stem(word))
    return words
    
def my_tokenizer(text):
#     tokens=nltk.tokenize.TreebankWordTokenizer().tokenize(text)
#     tokens=nltk.tokenize.word_tokenize(text)
    tokens =nltk.tokenize.wordpunct_tokenize(text)
    
    newtokens= stemWords(tokens)
    
    return newtokens

def convert_to_sparse_pandas(df, exclude_columns=[]):
    """
    Converts columns of a data frame into SparseArrays and returns the data frame with transformed columns.
    Use exclude_columns to specify columns to be excluded from transformation.
    :param df: pandas data frame
    :param exclude_columns: list
        Columns not be converted to sparse
    :return: pandas data frame
    """
    df = pd.DataFrame()
    exclude_columns = set(exclude_columns)

    for (columnName, columnData) in df.iteritems():
        if columnName in exclude_columns:
            continue
        df[columnName] = pd.arrays.SparseArray(columnData.values, dtype='uint8')

    return df

my_tokenizer(dataset['v2'][0])

In [20]:
vectorizer = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'),tokenizer=my_tokenizer,ngram_range=(1,1))
X = vectorizer.fit_transform(dataset['v2'])
tokens=vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [21]:
len(tokens)

5847

In [22]:
Labels = dataset['v1']
Data = pd.DataFrame(data=X.toarray(),columns=tokens)
SparseData = Data.astype(pd.SparseDtype(np.int16))


In [23]:
r = random.randint(0,Data.shape[1]-1)
print(r)
print(np.unique(Data.iloc[:,r]))
print(Data.memory_usage().sum()*pow(10,-6)*3, " mb")
print(dataset.memory_usage().sum()*pow(10,-6)*3," mb")
print(SparseData.memory_usage().sum()*pow(10,-6)*3," mb")



1961
[0 1]
724.513848  mb
0.37173599999999996  mb
0.679704  mb


In [24]:
encoder = LabelEncoder()
Labels = encoder.fit_transform(Labels)

In [25]:
print(Data.shape)
print(SparseData.shape)
print(dataset.shape)
print(Labels.shape)

(5163, 5847)
(5163, 5847)
(5163, 2)
(5163,)


Train model

In [26]:
from sklearn.metrics import classification_report


In [28]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(SparseData,
                        Labels, test_size = 0.30, random_state = 147)

Naive Bayes

In [29]:
from sklearn.naive_bayes import GaussianNB

In [30]:
data_train, data_test, target_train, target_test = train_test_split(Data,
                        Labels, test_size = 0.30, random_state = 147)

In [31]:
gnb = GaussianNB()
gnb.fit(data_train,target_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [32]:
target_pred = gnb.predict(data_test)

In [33]:
print(classification_report(target_test,target_pred))

              precision    recall  f1-score   support

           0       0.97      0.84      0.90      1339
           1       0.46      0.86      0.60       210

    accuracy                           0.85      1549
   macro avg       0.72      0.85      0.75      1549
weighted avg       0.90      0.85      0.86      1549



Decision Tree

In [34]:
from sklearn import tree

In [35]:
mytree = tree.DecisionTreeClassifier(random_state=147,max_depth=20,max_leaf_nodes=147)

In [36]:
mytree.fit(data_train,target_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=20, max_features=None, max_leaf_nodes=147,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=147, splitter='best')

In [37]:
target_pred=mytree.predict(data_test)

In [38]:
print(classification_report(target_test,target_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1339
           1       0.90      0.78      0.84       210

    accuracy                           0.96      1549
   macro avg       0.93      0.88      0.91      1549
weighted avg       0.96      0.96      0.96      1549



Optional Exercise: Try this on full spam.csv file and bigram matching instead of unigram matching

